# 导入tushare

In [ ]:
import tushare as ts
import pandas as pd
import decimal
from decimal import Decimal
from datetime import datetime
import time

# 初始化pro接口

In [ ]:
pro = ts.pro_api('ec30ba0eb5f07c64bf98162161fd2009bbd3ca48b7c02f3a81b99f97')

# 查询当前所有正常上市交易的股票列表

In [ ]:
stockBasicData = pro.stock_basic(exchange='', list_status='L')

In [ ]:
print(type(stockBasicData.columns.values),stockBasicData.columns.values)
#print(stockBasicData["industry"].value_counts())

# 获取当年交易日历

In [ ]:
today = (datetime.now().strftime('%Y%m%d'))
yearStart = ((datetime.now().strftime('%Y'))+"0101")
yearEnd = (datetime.now().strftime('%Y'))+"1231"
nextDay = ""
trade_daily = pro.trade_cal(exchange='', start_date=yearStart, end_date=yearEnd)

tradeCalendar=list()
for row in trade_daily.itertuples():
    if getattr(row, 'is_open') == 1:
        tradeCalendar.append(getattr(row, 'cal_date'))

yesterday = tradeCalendar[tradeCalendar.index(today) - 1]
print(yesterday)

# 获取股票所属市场

In [ ]:
def getMarketOfStock(ts_code):
    for row in stockBasicData.itertuples():
        if getattr(row, 'ts_code') == ts_code:
            return getattr(row, 'market')

# 获取股票基本信息

In [ ]:
def getBasicOfStock(ts_code):
    for row in stockBasicData.itertuples():
        if getattr(row, 'ts_code') == ts_code:
            area = getattr(row, 'area')
            industry = getattr(row, 'industry')
            name = getattr(row, 'name')
            return area, industry, name

# 获取涨停板系数

In [ ]:
def getIncrease(market):
    if market == '主板':
        return '1.1'
    elif market == '中小板':
        return '1.1'
    elif market == '创业板':
        return '1.2'
    elif market == '科创板':
        return '1.2'
    elif market == 'CDR':
        return '1.2'
    elif market == '北交所':
        return '1.3'
    else :
        return '0'
    

# 涨停板判断

In [ ]:
decimal.getcontext().rounding = "ROUND_HALF_UP"

def isGoodJobCal(openPrice,closePrice, increase):
    limitUp = (Decimal(str(openPrice))*Decimal(increase)).quantize(decimal.Decimal("0.00"))
    if Decimal(str(closePrice)) == limitUp:
        return True
    else :
        return False

# 计算涨停板

In [ ]:
def cal_banban_daily(data):
    banTotolAmount = 0
    marketTotolAmount = 0
    pd.set_option('display.max_rows', None)
    zhangtingdata = pd.DataFrame(columns=['ts_code', 'name', 'amount', 'area', 'industry'], index=[])

    for row in data.itertuples():
        marketTotolAmount +=getattr(row, 'amount') 
        if getattr(row, 'high') == getattr(row, 'close'):
            if isGoodJobCal(getattr(row, 'pre_close'), getattr(row, 'close'),getIncrease(getMarketOfStock(getattr(row, 'ts_code')))):
                banTotolAmount +=getattr(row, 'amount')
               # print("Good Job:",getattr(row, 'ts_code'),"deal amount:", round(getattr(row, 'amount')/100000, 3),"亿元")
                stockArea, stockIndustry, stockName = getBasicOfStock(getattr(row, 'ts_code'))
                size = zhangtingdata.index.size
                zhangtingdata.loc[size] = [getattr(row, 'ts_code'), stockName, getattr(row, 'amount'), stockArea, stockIndustry]
                #print("")
    print(zhangtingdata["industry"].value_counts())
    zhangtingdata.sort_values("industry",inplace=True)
    print(zhangtingdata)
    print("涨停板总量",round(banTotolAmount/100000, 3),"亿元")
    print("涨停量/市场成交量:", banTotolAmount/marketTotolAmount)
    return zhangtingdata
    #zhangtingdata.sort_values("amount",inplace=True)
    #print(zhangtingdata)




# 拉取数据

In [ ]:
tradeDay = today
#tradeDay = yesterday

oneDayData = pro.daily(**{
    "ts_code": "",
    "trade_date": tradeDay,
    "start_date": "",
    "end_date": "",
    "offset": "",
    "limit": ""
}, fields=[
    "ts_code",
    "trade_date",
    "open",
    "high",
    "low",
    "close",
    "pre_close",
    "change",
    "pct_chg",
    "vol",
    "amount"
])

print("today data")
td_zhangting = cal_banban_daily(oneDayData)


tradeDay = yesterday
oneDayData = pro.daily(**{
    "ts_code": "",
    "trade_date": tradeDay,
    "start_date": "",
    "end_date": "",
    "offset": "",
    "limit": ""
}, fields=[
    "ts_code",
    "trade_date",
    "open",
    "high",
    "low",
    "close",
    "pre_close",
    "change",
    "pct_chg",
    "vol",
    "amount"
])

print("yesterday data")
ytd_zhangting = cal_banban_daily(oneDayData)
#print(todayData)

In [ ]:
fff = td_zhangting.name.values
print(fff)

In [ ]:
count = 0
for f in fff:

    ggg = ytd_zhangting.where(ytd_zhangting["name"] == f).dropna()
    if(ggg.empty == False):
        count+=1
        print(f)
print(count)